# SQL
___

SQL (por sus siglas en inglés) es un lenguaje estructurado de consultas entre datos y algo llamado _"Relational Database"_. Este último termino se refiere a una colección especifica de tablas.

Cada tabla representa una entidad y está compuesta por atributos (columnas) y elementos (renglones). Los atributos de la siguiente tabla son `id`, `name`, `age` y `nationality`. Cada elemento (renglón) representa los datos personales de cada una de las siguientes personas.  

|id|name|age|nationality|
|--|----|---|-----------|
|1|Jessica|22|Ireland|
|2|Gabriel|48|France|
|3|Laura|36|USA|

Hay una gran cantidad de gestores de bases de datos (_DBMS_) relacionales uno de ellos es _PostgreSQL_ el cual es el más usado, de código abierto y además nos permite realizar y gestionar obejtos espaciales.

`psql` es el comando que nos ayuda comunicarnos desde el `shell` (_consola o terminal_) con _PostgreSQL_. De tal modo que si ejecutamos solamente estaremos interactuando con _PostgreSQL_ bajo un usuario `root` que por default es `postgres`:

```sql
$ psql -- desde una terminal

psql (11.1)
Type "help" for help.

postgres=# -- usuario root
```


En esta ocasión sera necesario trabajar dentro de nuestra base de datos `test` bajo el usuario `test`. Para ello será necesario abrir una terminal y ejecutar los siguientes comandos:

* `psql` para ingresar a _PostgreSQL_ y poder crear el usuario y la base de datos. Cabe mencionar que al ejecutar `psql` estamos en usuario _root_. Esto quiere decir que podemos hacer los cambios a las bases de datos y creación de otras desde el usuario `postgres`, puesto que es el usuario privilegiado y que tiene el control de todo.
* `CREATE USER test` creara el usuario `test` en el cual vivirán nuestras bases de datos posteriores.
* `CREATE DATABASE test OWNER test` creara la base de datos `test`, cuyo propietario será `test`.
* `ALTER ROLE test WITH SUPERUSER` con este comando le daremos el acceso total a nuestro usuario `test` para realizar todos los cambios necesarios dentro de la base de datos `test`.
* `CREATE SCHEMA sql` con este comando creamos los esquemas necesario, pero por esta ocasión uno en especial llamado `sql`. Será aquí donde guradaremos todas las tablas y datos generados en los ejercicios posteriores.

### Psycopg2

Psycopg es el adaptador más popular de PostgreSQL para Python, con el cual podemos convertir la interfaz de jupyter en PostgreSQL, realizando cualquier actividad como: creación, actualización y gestión de bases de datos.

Esta herramienta nos ayudará como canal para cmunicarnos desde Python con nuestras bases de datos y con ella poder realizar cualquier tipo de consulta.

Para ello necesitamos cargar la libreriía `psycopg2`, crear el objeto `conn = psycopg2.connect("dbname=test user=postgres")` para poder conectarnos con la base de datos de nuestra elección, luego el objeto `cur = conn.cursor()` que abre la comunicación con la base de datos. A partir de aquí lo siguiente es construir las consultas mediante el objeto `cur.execute`. Por ejemplo: `cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")` o `cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)", (100, "abc'def"))`.

Si queremos capturar la respuesta del _Query_ que hemos realizado a una base de datos y visualizarla como _output_ en jupyter, utilizamos el objeto `cur.fetchone()`. Por ejemplo: necesitamos obtener el resultado de la consulta `cur.execute("SELECT * FROM test;")`. Entonces bastará con agregar, al final del _statement_ anterior el objeto `cur.fetchone()`:

```sql
cur.execute("SELECT * FROM test;")
cur.fetchone()
(1, 100, "abc'def")
```
Para poder realizar los cambios, desde Python a una tabla en una base de datos de PostgreSQL, utilizamos el objeto `conn.commit()`. Y para finalizar la comunicación con la base de datos los objetos `cur.close()` y `conn.close()`.



### SELECT (_selección de columnas_).

Nos concentraremos en la consulta de la información a una tablas. Para ello es necesario contruir la sintaxis que realizará esta tarea, también conocido como _"queriying"_. 

Un _"Query"_ es la construcción de la sintaxis (_statement_), en _**SQL**_, para poder obtener una respuesta con los datos de una tabla o para generar nueva información o para actualizar una tabla preexistente. En _**SQL**_ podemos seleccionar una tabla usando las _keys words_ **SELECT** y **FROM** en mayúsculas como buena práctica. Por ejemplo, la siguiente sintaxis (_statement_) selecciona la columna `name` de la tabla `people`:

```sql
SELECT name FROM people;
```

Si utilizamos esquemas el _statement_ cambia: `SELECT tabla.columna FROM esquema.tabla;`. 

Para poder realizar la elección de de múltiples columnas bastará con especificar el nombre de cada una de ellas. Por ejemplo;

```sql
SELECT name, birthdate FROM people;
```

O en su lugar seleccionar todas los atributos de la tabla utilizando `*`:

```sql
SELECT * FROM people;
```

Para limitar el número de elementos (_renglones_) de una consulta utilizamos la _key word_ `LIMIT`:

```sql
SELECT * FROM people LIMIT 10;
```

Si el resultado de nuestra consulta presume duplicados, `DISTINCT` selecciona solo los records **únicos** en un atributo:

```sql
SELECT DISTINCT rol FROM roles;
```

Si quicieramos realizar el conteo de valores existentes en un atributo, `COUNT` retorna el número de elementos:

```sql
SELECT COUNT(*) FROM people;
```

Mientras tanto, si necesitamos realizar un conteo de valores únicos _**non-missing values**_, a diferencia de utilizar `*`, `COUNT(DISTINCT atributo)` nos ayuda a realizar esta tarea;

```sql
SELECT COUNT(DISTINCT birthdate) FROM people;
```

Para finalizar esta parte crearemos la conección desde _Python_ con la base de datos `test`, creada previamente, utilizando _**psycopg2**_. Es necesario crear dentro de la base de datos `test` un esquema llamado `postgis`. Hay que tomar muy en cuenta los siguientes parámetros: 

* `dbname=` base de datos destino.
* `user=` usuario de la base de datos.

El parámetro `host=/tmp/` se agrega por si en caso se presenta el error siguiente:

<img src="error_psycopg2.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

Para mantener la conección abierta con la base de datos ejecutamos, por separado la llamada de la librería **psycopg2** y la llamada de la base de datos:

In [2]:
import psycopg2.extras
conn = psycopg2.connect("dbname='test' user='test' host='/tmp/'")
cur = conn.cursor()

In [ ]:
cur.execute(
    """
    CREATE TABLE postgis.tabla_psycopg (id serial PRIMARY KEY, num integer, data varchar);
    INSERT INTO postgis.tabla_psycopg VALUES (1, 2, 'mi_primera_tabla');
    """
)
conn.commit()
cur.close()
conn.close()

Lo anterior habrá creado una tabla con los valores descritos dentro de la base de datos `test` en el esquema `test`.

```sql
SELECT title FROM films WHERE title = 'Metropolis';
```
Para este punto necesitamos manejar ciertas habilidades un poco avanzadas para poder trabajar con las tablas de los ejercicios. De tal motivo que se menciona, paso a paso, el procedimiento para realizar la carga de un archivo _.sql_ que contiene el script para crear las tablas dentro de una base de datos cuyo esquema nombraremos como `intro_sql`.

Es necesario recrear los siguientes comandos dentro de una terminal. También es necesario haber creado la base de datos anterior, nos referimos a `test` con el usuario `test`. 

* En una terminal ejecutamos el comando `psql test test` el cual nos conectara con la base de datos `test` y bajo el usuario `test`.
* Una vez conectados a la base de datos tiramos la línea `CREATE SCHEMA intro_sql`

Ahora ingresamos [dirección](https://assets.datacamp.com/production/repositories/641/datasets/d39d51ab6615b8a7f36681ee88490978fb095193/films.sql) y tenemos dos opciones para descargar el archivo: via `wget` (_terminal_) o dando click derecho y descargando el archivo _.sql_.

Con el archivo descargado ahora necesitamos ingresar a el para editar una línea. Podemos hacerlo desde terminal vía `gedit` o con algún editor de textos.

Dentro del archivo será necesario ubicar la línea `SET search_path = public, pg_catalog;` y colocar en `search_path = intro_sql`. Parte del archivo se muestra como sigue:

```sql
CREATE EXTENSION IF NOT EXISTS plpgsql WITH SCHEMA pg_catalog;


--
-- Name: EXTENSION plpgsql; Type: COMMENT; Schema: -; Owner:
--

COMMENT ON EXTENSION plpgsql IS 'PL/pgSQL procedural language';


SET search_path = intro_sql, pg_catalog;

SET default_tablespace = '';
```

Guardamos cambios y regresamos a una _terminal_ para poder ejecutar el comando `psql -U test test < /ruta/del/archivo/films.sql`.

Durante la ejecución del comando se arrojarán algunos errores como los siguientes, pero no hay por que hacer caso. La salida del comando será algo parecido a las siguientes líneas:

```sql
ERROR:  role "colinricardo" does not exist
CREATE TABLE
ERROR:  role "colinricardo" does not exist
CREATE TABLE
ERROR:  role "colinricardo" does not exist
CREATE TABLE
ERROR:  role "colinricardo" does not exist
COPY 4968
COPY 8397
COPY 4968
COPY 19791
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
```

Si ejecutamos el comando dentro `\dn` dentro de de la base de datos `test` veremos algo similar a esto:

```sql
List of relations
  Schema   |  Name   | Type  | Owner 
-----------+---------+-------+-------
 intro_sql | films   | table | test
 intro_sql | people  | table | test
 intro_sql | reviews | table | test
 intro_sql | roles   | table | test
```
las tablas han sido cargadas en el esquema `intro_sql`.

### Filtros con cantidades.

En **SQL**, la _keyword_ nos permite filtrar resultados ingresando texto o números. Hay diferente operadores de comparación como:

* `=` igual.
* `<>` no igual.
* `<` menos que.
* `>` mayor que.
* `<=` menor o igual.
* `>=` mayor que.

Por ejemplo: podemos filtrar de la tabla `films` y el título de la película `Metropolis`. Para este ejemplo regresaremos, en un _Dataframe_, la consulta realizada en **Posgres**. Será necesario importar la librería de _pandas_ y utilizar el método `extras`, de **psycopg2**, para convertir el **query** en un objeto de **python** en tabla.

`fetchall` nos regresa todos los elementos del **query** en el objeto `data` y es este el que, dentro de un `for`, imprime los elementos de la consulta. 

In [3]:
import pandas as pd

cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
cur.execute(
    """
    SELECT * FROM intro_sql.films WHERE title = 'Metropolis';
    """
)
data = cur.fetchall()
conn.commit()
cur.close()
pd.DataFrame([i.copy() for i in data])

,id,title,release_year,country,duration,language,certification,gross,budget
0,4,Metropolis,1927,Germany,145,German,Not Rated,26435,6000000


Para consultar el número de películas que se estrenaron antes del año 2000 la consulta sería de la sigueinte manera:
```sql
SELECT COUNT(*) FROM intro_sql.films WHERE release_year<2000
```

La consulta para ver todos los detalles de las películas estrenadas en el 2016
```sql
SELECT * FROM intro_sql.films WHERE release_year = 2016
```

Para conocer el título y el año de las películas estrenadas despues del año 2000
```sql
SELECT title, release_year FROM intro_sql.films WHERE release_year>2000
```


In [ ]:
release_year